In [5]:
import time, os, threading, cv2
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime

In [2]:
def scrape(path, headless=False , id=0):


    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--hide-scrollbars")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(path)
    driver.maximize_window()
    
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler")))
        cookie.click()
    except:
        print("No cookie")
    
    try:
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@id="loginBtn"]')))
        login.click()

        user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "Username")))
        user_name.send_keys("sultanmirza0501@gmail.com")

        password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "Password")))
        password.send_keys("Muhssan7865")

        login_tab = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Login']")
        driver.execute_script("arguments[0].scrollIntoView();", login_tab)
        login_tab.click()
        driver.back()
    except Exception:
        print("No login required or login failed.")
        
    
    # Navigate to Live Auctions
    try:
        # auc = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//a[text()="Auctions"])[1]')))
        # auc.click()

        # lives = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text()="Open LIVE"]')))
        # lives[0].click()

        # WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="button full-width mb-0"]'))).click()
        driver.get(f"https://www.astonbarclay.net/FMLiveToken/?saleid={id}&hall=")

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/a[@class="button full-width mb-0"]'))
        ).click()
    except Exception:
        print("Live auction not found.")
    
    time.sleep(5)
    return driver

# def record_website_screen(driver, output_file="live_rec.avi", fps=8):
#     try:
#         img = driver.get_screenshot_as_png()
#         nparr = np.frombuffer(img, np.uint8)
#         frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#         height, width, _ = frame.shape
#         fourcc = cv2.VideoWriter_fourcc(*'XVID')
#         out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

#         frame_count = 0
#         while True:
#             img = driver.get_screenshot_as_png()
#             nparr = np.frombuffer(img, np.uint8)
#             frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#             frame_count += 1
#             out.write(frame)
#             time.sleep(1/fps)
#     except Exception as e:
#         print(f"Error in screen recording: {e}")
#     finally:
#         out.release()
#         print(f"Recording saved, captured {frame_count} frames.")

# def start_recording(driver):
#     thread = threading.Thread(target=record_website_screen, args=(driver,))
#     thread.daemon = True
#     thread.start()
def save_full_screenshot(driver, prefix="bidding_status"):
    try:
        os.makedirs("screenshots", exist_ok=True)
        filename = f"screenshots/{prefix}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")

if __name__ == "__main__":
    path = "https://www.astonbarclay.net/auctions"
    driver = scrape(path, headless=False,id=17580)
    print("Starting screen recording...")
    # start_recording(driver)
    
    print("Starting data extraction...")
    results = []
    try:
        registration = ""
        while True:
            if not driver.window_handles:
                print("Browser window closed. Stopping script.")
                break
            details = {}
            try:
                try:
                    Reg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="currentLot-title"])[2]')))
                    curr_reg = Reg.text.strip()
                    
                    if curr_reg != registration:
                        registration = curr_reg
                        details['Reg'] = curr_reg.split(" - ")[1]
                        
                        
                    try:
                        data_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="bidding-history-messages "]')))
                        prices_elements = WebDriverWait(data_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="d-flex flex-row align-items-center history-text wordwrap"]')))
                        price = [p.text.strip() for p in prices_elements]
                        details['Bidding History'] = price
                        for text in price:
                            if "SOLD" in text.upper():
                                save_full_screenshot(driver, prefix=registration)
                                break
                    except Exception as e:
                        print(f"Error getting prices")
                    
                except Exception as e:
                    print(f"Error getting Reg:")
                except StaleElementReferenceException:
                    print("Stale element, retrying...")
                    continue
            except:
                pass
            time.sleep(2)

            if details:
                # Write to CSV incrementally
                results.append(details)
                df = pd.DataFrame(results, columns=["Reg", "Bidding History"])
                # header = not os.path.exists("live_data.csv")
                df.to_csv("live_data.csv", mode='a', header = not os.path.exists("live_data.csv"), index=False)
            
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        driver.quit()
        print("Browser closed.")


Live auction not found.
Starting screen recording...
Starting data extraction...
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Screenshot saved: screenshots/2 - SD67YSS.png
📸 Scree

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x96fe43+66515]
	GetHandleVerifier [0x0x96fe84+66580]
	(No symbol) [0x0x75da6b]
	(No symbol) [0x0x79b5ab]
	(No symbol) [0x0x7cb086]
	(No symbol) [0x0x7c667c]
	(No symbol) [0x0x7c5bf3]
	(No symbol) [0x0x72e72d]
	(No symbol) [0x0x72ecae]
	(No symbol) [0x0x72f14d]
	GetHandleVerifier [0x0xbc7353+2521315]
	GetHandleVerifier [0x0xbc22d3+2500707]
	GetHandleVerifier [0x0x997c94+229924]
	GetHandleVerifier [0x0x9881f8+165768]
	GetHandleVerifier [0x0x98ecad+193085]
	(No symbol) [0x0x72e3e3]
	(No symbol) [0x0x72db60]
	GetHandleVerifier [0x0xd0c20f+3852191]
	BaseThreadInitThunk [0x0x76cbfcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x77da82ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x77da827e+238]


In [3]:
df = pd.read_csv("live_data.csv")

df2 = df.drop_duplicates(subset=['Reg'], keep='first')

finalized_df = df2.to_csv("live_data_CLEANED.csv", index=False)
# delete the original df
os.remove("live_data.csv")

In [43]:
import pandas as pd

# Load CSV
df = pd.read_csv("live_data_CLEANED.csv")

# Keep only rows where 'Reg' is not empty or NaN
df_cleaned = df[df['Reg'].notna() & (df['Reg'] != '')].copy()

# Save back to CSV (overwrite original)
df_cleaned.to_csv("live_data_CLEANED.csv", index=False)

print("✅ Rows with empty 'Reg' deleted and CSV updated.")


✅ Rows with empty 'Reg' deleted and CSV updated.


In [44]:
import pandas as pd
import ast

# CSV load karo
df = pd.read_csv("live_data_CLEANED.csv", index_col=0, header=None, names=["Reg", "Bidding History"])

# Convert string representation of list to actual list safely
def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

df["Bidding History"] = df["Bidding History"].apply(safe_eval)

# Shift the data
shifted = df["Bidding History"].shift(-1)  # i -> i+1
df["Bidding History"] = shifted.fillna('').tolist()  # last row ko empty list

# Save CSV
df.to_csv("live_data_SHIFTED.csv", index=True, header=False)


In [45]:
import pandas as pd
import ast
import os

# Pehle live_data_CLEANED.csv ko delete kar do
if os.path.exists("live_data_CLEANED.csv"):
    os.remove("live_data_CLEANED.csv")

# Shifted CSV load karo
df_shifted = pd.read_csv("live_data_SHIFTED.csv", header=None, names=["Reg", "Bidding History"])

# First row skip karo
df_shifted = df_shifted.iloc[1:].reset_index(drop=True)

# Agar "Bidding History" string me hai to convert list
def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

df_shifted["Bidding History"] = df_shifted["Bidding History"].apply(safe_eval)

# Save new cleaned CSV with 2 columns
df_shifted.to_csv("live_data_CLEANED.csv", index=False, columns=["Reg", "Bidding History"])

os.remove("live_data_SHIFTED.csv")


In [46]:
import pandas as pd
import ast

# CSV load
df = pd.read_csv("live_data_CLEANED.csv")

def remove_now_offering(history):
    try:
        lst = ast.literal_eval(history) if isinstance(history, str) else history
    except:
        lst = []
    # Keep only items not starting with 'NOW OFFERING: LOT'
    new_list = [item for item in lst if not (isinstance(item, str) and item.startswith('NOW OFFERING: LOT'))]
    return new_list

df['Bidding History'] = df['Bidding History'].apply(remove_now_offering)

# Save CSV
df.to_csv("live_data_CLEANED.csv", index=False)


In [49]:
import pandas as pd
import ast
import re

# CSV load
df = pd.read_csv("live_data_CLEANED.csv")

def keep_current_lot(history_list):
    try:
        lst = ast.literal_eval(history_list) if isinstance(history_list, str) else history_list
    except:
        lst = []

    cleaned_list = []
    current_lot_found = False
    for entry in lst:
        if isinstance(entry, str):
            # Agar ye current lot (first LOT in list) ya uske continuation hai
            if not current_lot_found:
                if re.search(r'LOT \d+ SOLD TO', entry) or re.search(r'LOT \d+ PROVISIONALLY SOLD', entry):
                    current_lot_found = True
                    cleaned_list.append(entry)
            else:
                # Agar ye purana lot ka entry hai, stop
                if re.search(r'LOT \d+ PROVISIONALLY SOLD', entry) or re.search(r'NOW OFFERING: LOT', entry):
                    break
                cleaned_list.append(entry)
    return cleaned_list

# Apply
df['Bidding History'] = df['Bidding History'].apply(keep_current_lot)

# Save cleaned CSV
df.to_csv("live_data_CLEANED.csv", index=False)

# Preview
print(df['Bidding History'].head())


0    [LOT 1 PROVISIONALLY SOLD TO HALL £5,150, HALL...
1    [LOT 2 SOLD TO HALL FOR £5,950, HALL: £5,950, ...
2    [LOT 3 SOLD TO LIVE FOR £12,800, NOW TAKING BI...
3    [LOT 4 SOLD TO LIVE FOR £6,100, NOW TAKING BID...
4    [LOT 5 PROVISIONALLY SOLD TO LIVE £2,800, NOW ...
Name: Bidding History, dtype: object


In [51]:
df1 = pd.read_csv("live_data_CLEANED.csv")

df2 = pd.read_csv("barclay_renamed.csv")

merged_df = df2.merge(df1[['Reg',"Bidding History"]], on='Reg', how='left')


output_path = "barclay_live_data_merged_final.csv"
merged_df.to_csv(output_path, index=False)

df = pd.read_csv("barclay_live_data_merged_final.csv")

df.to_csv("Final_Barclay.csv", index=False)


In [52]:
import pandas as pd
import ast
import re

df = pd.read_csv("Final_Barclay.csv")

def extract_status_and_bid(history_list):
    # Convert string to list
    try:
        lst = ast.literal_eval(history_list) if isinstance(history_list, str) else history_list
    except:
        lst = []

    status = "NOT SOLD"
    last_bid = ""

    for entry in lst:
        if isinstance(entry, str):
            # Check PROVISIONALLY
            if "PROVISIONALLY SOLD" in entry:
                status = "PROVISIONALLY"
                match = re.search(r'£[\d,]+', entry)
                if match:
                    last_bid = match.group(0)
                break
            # Check SOLD
            elif "SOLD TO" in entry:
                status = "SOLD"
                match = re.search(r'£[\d,]+', entry)
                if match:
                    last_bid = match.group(0)
                break

    return pd.Series([status, last_bid])

# Apply extraction
df[['bidding_status', 'last_bid']] = df['Bidding History'].apply(extract_status_and_bid)

# Save CSV
df.to_csv("Final_Barclay_cleaned.csv", index=False)

# Preview
print(df[['Bidding History', 'bidding_status', 'last_bid']].head())


                                     Bidding History bidding_status last_bid
0  ['LOT 1 PROVISIONALLY SOLD TO HALL £5,150', 'H...  PROVISIONALLY   £5,150
1  ['LOT 2 SOLD TO HALL FOR £5,950', 'HALL: £5,95...           SOLD   £5,950
2  ['LOT 3 SOLD TO LIVE FOR £12,800', 'NOW TAKING...           SOLD  £12,800
3  ['LOT 4 SOLD TO LIVE FOR £6,100', 'NOW TAKING ...           SOLD   £6,100
4  ['LOT 5 PROVISIONALLY SOLD TO LIVE £2,800', 'N...  PROVISIONALLY   £2,800
